## Functions required for the code

__Nested Key Finding Function__

In [16]:
def nest_key(d, indent=0):
    
    for key, value in d.items():
        
        print('\t' * indent + f'{key}')
        
        if isinstance(value, dict):
            
            nest_key(value, indent+1)

__Interpolation Function__

In [17]:
from scipy.interpolate import interp1d

In [18]:
def interp_ocv(z,OCV):
    
    dum = interp1d(z,OCV)
    
    return dum

__OCV calculation for any SOC value__

In [19]:
def OCV_from_SOC(data,OCV_0,OCV_rel,temp,eta,Q):

    etaik = data['S1']['current']

    # Negative current tends to increase the SOC thus it is charging current
    etaik[etaik < 0] = eta*etaik[etaik < 0]

    z = 1 - np.cumsum(etaik)*1/(Q*3600)  

    # OCV calculation from the given SOC (Coloumb Counting)

    OCV_0_z = interp_ocv(SOC,OCV_0)(z)
    OCV_rel_z = interp_ocv(SOC,OCV_rel)(z)

    tmp = int(temp[-2:])

    if (temp[-3] == 'N'):

        tmp = -tmp

    tmp

    ocv_z = OCV_0_z + tmp*OCV_rel_z

    return (ocv_z,z)

## Main Code

__File Input__

In [20]:
d = pd.read_pickle('A123_OCV_Dynamic.pkl')
stat_model = pd.read_pickle('Static Model.pkl')

In [21]:
nest_key(d)

10_N25
	S1
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S2
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S3
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
10_N15
	S1
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S2
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S3
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
30_N05
	S1
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S2
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S3
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
45_P05
	S1
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S2
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S3
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
45_P15
	S1
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S2
		time
		step
		current
		voltage
		charge_cap
		discharge_cap
	S3
		time
		step
		current

In [22]:
mag_list = list(d.keys())
script_list = list(d[mag_list[0]].keys())

In [23]:
mag_list

['10_N25',
 '10_N15',
 '30_N05',
 '45_P05',
 '45_P15',
 '50_P25',
 '50_P35',
 '50_P45']

__Model Input Variables__

In [24]:
stat_model.keys()

dict_keys(['OCV_0', 'OCV_rel', 'SOC', 'v', 'SOC_0', 'SOC_rel'])

In [25]:
# Static model parameters

SOC = stat_model['SOC']
OCV_0 = stat_model['OCV_0'] 
OCV_rel = stat_model['OCV_rel']


data_25 = d['50_P25']
total_dis_cap = 0
total_chg_cap = 0
out_d = {}

___Calculations at 25°C___

In [26]:
# Total Discharge Capacity

for i in script_list:
    
    total_dis_cap = total_dis_cap + np.array(data_25[i]['discharge_cap'])[-1]
    
# total Charging Capacity

for i in script_list:
    
    total_chg_cap = total_chg_cap + np.array(data_25[i]['charge_cap'])[-1]

# Coloumb efficiency calculation

eta25 = total_dis_cap/total_chg_cap

# Adjusting the charge capacity 

for i in script_list:
    
    data_25[i]['charge_cap'] = eta25*data_25[i]['charge_cap']
    
# Total battery capacity 
# Should be same at all the temperatures if battery degradation is not present

Q = (np.array(data_25['S1']['discharge_cap'])[-1] + \
    np.array(data_25['S2']['discharge_cap'])[-1] - \
    np.array(data_25['S1']['charge_cap'])[-1] - \
    np.array(data_25['S2']['charge_cap'])[-1])

# OCV calculation for test SOC

(ocv_z,z) = OCV_from_SOC(data_25,OCV_0,OCV_rel,'50_P25',eta25,Q)

# Storing the data in the dictionary

out_d['50_P25'] = {}
out_d['50_P25']['eta'] = eta25
out_d['50_P25']['Q']  = Q

___Calculation at any temperature___

In [27]:
temp_list = mag_list
temp_list.remove('50_P25')

In [28]:
temp_list

['10_N25', '10_N15', '30_N05', '45_P05', '45_P15', '50_P35', '50_P45']

In [29]:
for temp in temp_list:

    data = d[temp]
    total_dis_cap = 0
    total_chg_cap = 0


    data['S2']['charge_cap'] = eta25*data['S2']['charge_cap']
    data['S3']['charge_cap'] = eta25*data['S3']['charge_cap']

    # Total Discharge Capacity

    for i in script_list:

        total_dis_cap = total_dis_cap + np.array(data[i]['discharge_cap'])[-1]

    total_dis_cap = total_dis_cap - np.array(data['S2']['charge_cap'])[-1] - np.array(data['S3']['charge_cap'])[-1]

    # total Charging Capacity

    for i in script_list:

        total_chg_cap = total_chg_cap + np.array(data[i]['charge_cap'])[-1]

    total_chg_cap = total_chg_cap - np.array(data['S2']['charge_cap'])[-1] - np.array(data['S3']['charge_cap'])[-1]

    # Coloumb efficiency calculation

    eta = total_dis_cap/total_chg_cap

    # Adjusting the charge capacity 

    for i in ['S1','S3']:

        data[i]['charge_cap'] = eta*data[i]['charge_cap']

    Q = (np.array(data['S1']['discharge_cap'])[-1] + \
           np.array(data['S2']['discharge_cap'])[-1] - \
           np.array(data['S1']['charge_cap'])[-1] - \
           np.array(data['S2']['charge_cap'])[-1])
    
    
    # OCV calculation for test SOC

    (ocv_z,z) = OCV_from_SOC(data,OCV_0,OCV_rel,temp,eta,Q)
    
    # Storing the data in the dictionary

    out_d[temp] = {}
    out_d[temp]['eta'] = eta
    out_d[temp]['Q']  = Q
    

In [30]:
out_d

{'50_P25': {'eta': 0.9944503313637096, 'Q': 2.0495322455503873},
 '10_N25': {'eta': 0.935275088307756, 'Q': 2.0559745289965687},
 '10_N15': {'eta': 0.9687882710682045, 'Q': 2.0572662044936147},
 '30_N05': {'eta': 0.9892038434260564, 'Q': 2.063050665421433},
 '45_P05': {'eta': 0.995362805532912, 'Q': 2.046536130318432},
 '45_P15': {'eta': 0.9952676564918135, 'Q': 2.0518072720415232},
 '50_P35': {'eta': 0.99366379588643, 'Q': 2.0498289157492047},
 '50_P45': {'eta': 0.992410804916624, 'Q': 2.0432672084627956}}